In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.interpolate import interp1d

# Initialize parameters
A_bar = 1.0      # Fixed productivity level
theta = 0.5      # Elasticity of output with respect to capital
p = 1.0          # Price of capital goods
gamma = 0.1      # Adjustment cost parameter
beta = 0.95      # Discount factor

# Profit function Π(K)
def profit_function(K):
    return np.exp(A_bar) * (K ** theta)

# Cost function C(I, K)
def cost_function(I):
    return (gamma / 2) * (I ** 2)

# Right-hand side of the Bellman equation
def bellman_rhs(K, I, V_next):
    return profit_function(K) - p * I - cost_function(I) + beta * V_next

# Bellman equation with interpolation for V_func
def bellman_equation(K, V_func):
    # Interpolation function for V_func
    K_grid = np.array(list(V_func.keys()))
    V_values = np.array(list(V_func.values()))
    V_interp = interp1d(K_grid, V_values, fill_value="extrapolate")

    # Define objective function
    def objective(I):
        K_next = K + I
        V_next = V_interp(K_next)  # Interpolated value
        return -bellman_rhs(K, I, V_next)  # Maximization as minimization

    # Solve for optimal investment I_t
    result = minimize(objective, x0=0.1, bounds=[(0, None)])
    if result.success:
        I_opt = result.x[0]
    else:
        I_opt = 0

    K_next = K + I_opt
    V_next = V_interp(K_next)
    V = bellman_rhs(K, I_opt, V_next)
    return V

# Initialize value function as a dictionary
value_function = {K: profit_function(K) / (1 - beta) for K in np.linspace(0.1, 2, 10)}

# Iterative computation of the value function
for iteration in range(100):
    new_value_function = {}

    # Update value function for each capital level in the grid
    for K in np.linspace(0.1, 2, 10):
        new_value_function[K] = bellman_equation(K, value_function)
    
    # Check for convergence
    if all(abs(new_value_function[K] - value_function[K]) < 1e-4 for K in new_value_function):
        print(f"Converged after {iteration + 1} iterations")
        break
    
    # Update value function
    value_function = new_value_function

# Display final value function
for K, V in value_function.items():
    print(f"K = {K:.2f}, V = {V:.4f}")


K = 0.10, V = 165118036.5761
K = 0.31, V = 165118035.8473
K = 0.52, V = 165118038.4687
K = 0.73, V = 165118038.6947
K = 0.94, V = 165118039.4064
K = 1.16, V = 165118039.8049
K = 1.37, V = 165118040.4316
K = 1.58, V = 165118041.1402
K = 1.79, V = 165118041.7818
K = 2.00, V = 165117700.5137
